In [ ]:
import pandas as pd
df = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
df.head()

In [ ]:
print(f"Shape - {df.shape}")
print(f"columns - {df.columns}")

In [ ]:
import numpy as np
from collections import Counter
import string
import re
from tqdm import tqdm
from textblob import TextBlob
from urllib.error import HTTPError
import nltk
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag


In [ ]:
#preprocessing:
punc = string.punctuation + '•'
contractions_dict = {"ain't": 'are not',
 "'s": ' is',
 "aren't": 'are not',
 "can't": 'cannot',
 "can't've": 'cannot have',
 "'cause": 'because',
 "could've": 'could have',
 "couldn't": 'could not',
 "couldn't've": 'could not have',
 "didn't": 'did not',
 "doesn't": 'does not',
 "don't": 'do not',
 "hadn't": 'had not',
 "hadn't've": 'had not have',
 "hasn't": 'has not',
 "haven't": 'have not',
 "he'd": 'he would',
 "he'd've": 'he would have',
 "he'll": 'he will',
 "he'll've": 'he will have',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how'll": 'how will',
 "i'd": 'i would',
 "i'd've": 'i would have',
 "i'll": 'i will',
 "i'll've": 'i will have',
 "i'm": 'i am',
 "i've": 'i have',
 "isn't": 'is not',
 "it'd": 'it would',
 "it'd've": 'it would have',
 "it'll": 'it will',
 "it'll've": 'it will have',
 "let's": 'let us',
 "ma'am": 'madam',
 "mayn't": 'may not',
 "might've": 'might have',
 "mightn't": 'might not',
 "mightn't've": 'might not have',
 "must've": 'must have',
 "mustn't": 'must not',
 "mustn't've": 'must not have',
 "needn't": 'need not',
 "needn't've": 'need not have',
 "o'clock": 'of the clock',
 "oughtn't": 'ought not',
 "oughtn't've": 'ought not have',
 "shan't": 'shall not',
 "sha'n't": 'shall not',
 "shan't've": 'shall not have',
 "she'd": 'she would',
 "she'd've": 'she would have',
 "she'll": 'she will',
 "she'll've": 'she will have',
 "should've": 'should have',
 "shouldn't": 'should not',
 "shouldn't've": 'should not have',
 "so've": 'so have',
 "that'd": 'that would',
 "that'd've": 'that would have',
 "there'd": 'there would',
 "there'd've": 'there would have',
 "they'd": 'they would',
 "they'd've": 'they would have',
 "they'll": 'they will',
 "they'll've": 'they will have',
 "they're": 'they are',
 "they've": 'they have',
 "to've": 'to have',
 "wasn't": 'was not',
 "we'd": 'we would',
 "we'd've": 'we would have',
 "we'll": 'we will',
 "we'll've": 'we will have',
 "we're": 'we are',
 "we've": 'we have',
 "weren't": 'were not',
 "what'll": 'what will',
 "what'll've": 'what will have',
 "what're": 'what are',
 "what've": 'what have',
 "when've": 'when have',
 "where'd": 'where did',
 "where've": 'where have',
 "who'll": 'who will',
 "who'll've": 'who will have',
 "who've": 'who have',
 "why've": 'why have',
 "will've": 'will have',
 "won't": 'will not',
 "won't've": 'will not have',
 "would've": 'would have',
 "wouldn't": 'would not',
 "wouldn't've": 'would not have',
 "y'all": 'you all',
 "y'all'd": 'you all would',
 "y'all'd've": 'you all would have',
 "y'all're": 'you all are',
 "y'all've": 'you all have',
 "you'd": 'you would',
 "you'd've": 'you would have',
 "you'll": 'you will',
 "you'll've": 'you will have',
 "you're": 'you are',
 "you've": 'you have',
' afaik ': ' as far as i know ',
 ' afk ': ' away from keyboard',
 ' asap ': ' as soon as possible',
 ' atk ': ' at the keyboard',
 ' atm ': ' at the moment',
 ' bak ': ' back at keyboard ',
 ' bbl ': ' be back later ',
 ' bbs ': ' be back soon ',
 ' bfn ': ' bye for now ',
 ' b4n ': ' bye for now ',
 ' brb ': ' be right back ',
 ' brt ': ' be right there ',
 ' btw ': ' by the way ',
 ' b4 ': ' before ',
 ' cu ': ' see you ',
 ' cul8r ': ' see you later ',
 ' cya ': ' see you ',
 ' faq ': ' frequently asked questions ',
 ' fc ': ' fingers crossed ',
 ' fwiw ': " for what it is worth ",
 ' fyi ': ' for your information ',
 ' gal ': ' get a life ',
 ' gg ': ' good game ',
 ' gn ': ' good night ',
 'gmta': 'great minds think alike',
 ' gr8 ': ' great! ',
 ' g9 ': ' genius ',
 ' ic ': ' i see ',
 ' icq ': ' i seek you ',
 'ilu': ' i love you',
 'imho': ' in my humble opinion',
 ' imo ': ' in my opinion ',
 ' iow ': ' in other words ',
 'irl': 'in real life',
 ' ldr ': ' long distance relationship ',
 'lmao': ' laugh my ass off',
 ' lol ': ' laughing out loud ',
 'ltns': ' long time no see',
 ' l8r ': ' later ',
 ' mte ': ' my thoughts exactly ',
 ' m8 ': ' mate ',
 ' nrn ': ' no reply necessary ',
 ' oic ': ' oh i see ',
 ' pita ': ' pain in the ass ',
 ' prt ': ' party ',
 ' prw ': ' parents are watching ',
 'qpsa?': 'que pasa?',
 'rofl': 'rolling on the floor laughing',
 'roflol': 'rolling on the floor laughing out loud',
 'rotflmao': 'rolling on the floor laughing my ass off',
 ' sk8 ': ' skate ',
 ' stats ': ' your sex and age ',
 ' asl ': ' age, sex, location ',
 ' thx ': ' thank you ',
 'ttfn': 'ta-ta for now!',
 'ttyl': ' talk to you later',
 ' u ': ' you ',
 'u2': ' you too',
 'u4e': ' yours for ever',
 ' wb ': ' welcome back ',
 'wtf': ' what the fuck',
 'wtg': ' way to go!',
 'wuf': ' where are you from?',
 ' w8 ': ' wait ',
' pov ':' point of view '}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)


def text_cleaning(txt):
    
    txt = txt.lower()
    
    #html
    txt = ' '.join(['' if 'http' in words or 'www' in words or '.jpeg' in words or '.jpg' in words or '.png' in words or '.zip' in words or 'wikipeida_talk' in words or '@' in words or '•' in words or '&' in words or '\n' in words else words for words in txt.split() ]) #html .jpg .png zip wikipeida_Talk removal    
      
    #mail
    pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    match = re.findall(pattern, 'This is my mail karthik123@gmail.com note it')
    txt = txt.replace(match[0],'') #mail removal code
    
    txt = ''.join([char for char in txt if char not in punc+'1234567890' ]) #Punctuation & IP removal
    txt = re.sub(r'([*!?\'])\1\1{2,}',r'\1\1\1',txt)
    txt = re.sub(r'([*!?\']+)',r' \1 ',txt)
    txt = re.sub(r'([a-zA-Z])\1{2,}\b',r'\1\1', txt)    
    txt = re.sub(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1',txt)
    
    # Expanding Contractions in the comments
    txt = expand_contractions(txt)
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    txt = emoji_pattern.sub(r'', txt)
    #lang = TextBlob(txt) 
    
    #try:
      #  if lang.detect_language() != 'en' and len(txt) < 4000 :
       #     txt = GoogleTranslator(source='auto', target='en').translate(txt)
            
    #except HTTPError:
     #   print(txt)
        #txt = translation.text

    return txt

In [ ]:
tqdm.pandas()
df['cleaned_text'] = df['comment_text'].progress_apply(lambda x:text_cleaning(x))

In [ ]:
df.head()

In [ ]:
def tokenize(text):
        token = nltk.word_tokenize(text)
        return token

df['Tokenized_text'] = df['cleaned_text'].progress_apply(lambda x : tokenize(x))

stpw = nltk.corpus.stopwords.words('english')
def rmstp(text):
    tt = [char.lower() for char in text if char not in stpw ]
    return tt
df['Tokenized_text'] = df['Tokenized_text'].progress_apply(lambda x:rmstp(x))
df.head(10)

In [ ]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
def rmstmer(text):
    tt = [lemmatizer.lemmatize(char,tag_map[j[0]]) for char,j in pos_tag(text)]
    return tt
df['lemmatized'] = df['Tokenized_text'].progress_apply(lambda x:rmstmer(x))
print(df.head())

In [ ]:
df['lemmatized_text'] = [" ".join(i) for i in df['lemmatized']]
df.head()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
def word_cloud_generator(df):
    comment_words = ''
    stopwords = set(STOPWORDS)

    # iterate through the csv file
    for val in df.lemmatized_text:

        # typecaste each val to string
        val = str(val)

        # split the value
        tokens = val.split()

        # Converts each token into lowercase
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()

        comment_words += " ".join(tokens)+" "

    wordcloud = WordCloud(width = 800, height = 800,
                    background_color ='white',
                    stopwords = stopwords,
                    min_font_size = 10).generate(comment_words)

    # plot the WordCloud image                      
    plt.figure(figsize = (10, 10), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)
    plt.show()

In [ ]:
#'toxic', 'severe_toxic', 'obscene', 'threat',
#       'insult', 'identity_hate'
df_toxic = df[df['toxic'] == 1]
df_severe_toxic = df[df['severe_toxic'] == 1]
df_obscene = df[df['obscene'] == 1]
df_threat = df[df['threat'] == 1]
df_insult = df[df['insult'] == 1]
df_identity_hate = df[df['identity_hate'] == 1]

In [ ]:
#print("Category - Toxic")
word_cloud_generator(df_toxic)

In [ ]:
#print("Category - Severe Toxic")
word_cloud_generator(df_severe_toxic)

In [ ]:
#print("Category - Obscene")
word_cloud_generator(df_obscene)

In [ ]:
#print("Category - threat")
word_cloud_generator(df_threat)

In [ ]:
#print("Category - insult")
word_cloud_generator(df_insult)

In [ ]:
#print("Category - identity_hate")
word_cloud_generator(df_identity_hate)